In [1]:
# basic python data handling analysis modules
import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
sns.set()
import pickle, os, gc, re
# small utility functions
from utility import *

# interactive jupyter widgets!
# https://towardsdatascience.com/interactive-controls-for-jupyter-notebooks-f5c94829aee6
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

%matplotlib inline

In [2]:
# import warnings
# warnings.filterwarnings('ignore')
# # cleanliness!

In [3]:
import shutil
import tempfile
import urllib.request
import gzip
import datetime
import json
import time

Dataset_directory = ".."+os.sep+"Datasets"+os.sep
tmdb_dir = create_subdir(Dataset_directory, "themoviedb")

In [164]:
# import datetime
# datetime.datetime.now()

In [324]:
# https://developers.themoviedb.org/3/getting-started/daily-file-exports

# The export job runs every day starting at around 7:00 AM UTC, and all files are available by 8:00 AM UTC.

# http://files.tmdb.org/p/exports/movie_ids_04_28_2017.json.gz


# Here's something that would have been useful to know ahead of time:
# https://developers.themoviedb.org/3/getting-started/append-to-response

In [10]:
# import datetime

# utc_now = datetime.datetime.utcnow()

# if utc_now.hour<8: # upload for 'today' not necessary donw yet
#     utc_now = today - datetime.timedelta(days = 1)
    
# movie_ids_file = "http://files.tmdb.org/p/exports/movie_ids_"+utc_now.strftime("%m_%d_%Y")+".json.gz"
# movie_ids_file

In [9]:

utc_now = datetime.datetime.utcnow()

if utc_now.hour<8: # upload for 'today' not necessary donw yet
    utc_now = today - datetime.timedelta(days = 1)
    
daily_file_export_list = ["movie","tv_series","person","collection","tv_network","keyword","production_company"]

for daily_export in daily_file_export_list:

    daily_file_export_file = "http://files.tmdb.org/p/exports/"+daily_export+"_ids_"+utc_now.strftime("%m_%d_%Y")+".json.gz"
    # movie_ids_file

    outFilePath = tmdb_dir + daily_file_export_file.split("/")[-1].replace(".gz","")

    with urllib.request.urlopen(daily_file_export_file) as response:
        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
            shutil.copyfileobj(response, tmp_file)

    with gzip.open(tmp_file.name, mode='rt', encoding="utf-8") as f_in:
        with open(outFilePath, 'w', encoding="utf-8") as f_out:
            shutil.copyfileobj(f_in, f_out)

    daily_file_export_ids = pd.read_json(outFilePath, lines=True)
    daily_file_export_ids.describe(include='all')        

In [317]:
outFilePath = tmdb_dir + "movie_ids_06_29_2019.json"

movie_ids = pd.read_json(outFilePath, lines=True)
movie_ids.describe(include='all')

,adult,id,original_title,popularity,video
count,467797,467797.000000,467797,467797.000000,467797
unique,2,NaN,430051,NaN,2
top,False,NaN,Carmen,NaN,False
freq,443576,NaN,46,NaN,445325
mean,NaN,332134.697503,NaN,1.118390,NaN
std,NaN,175741.448301,NaN,1.995175,NaN
min,NaN,2.000000,NaN,0.000000,NaN
25%,NaN,191544.000000,NaN,0.600000,NaN
50%,NaN,347182.000000,NaN,0.600000,NaN
75%,NaN,482138.000000,NaN,0.841000,NaN


In [83]:
!pip install tmdbsimple

In [ ]:
# http://api.themoviedb.org/3/movie/16535?api_key=###

In [165]:
# import tmdbsimple as tmdb
text_file = open(tmdb_dir+"my_api_key.txt", "r")
# tmdb.API_KEY = text_file.readlines()[0]
api_key = text_file.readlines()[0]

In [98]:
# with urllib.request.urlopen("http://api.themoviedb.org/3/movie/16535?api_key="+tmdb.API_KEY) as response:
#     resp = response.readlines()
# #     with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
# #         shutil.copyfileobj(response, tmp_file)

In [155]:


tmdb_df = pd.DataFrame(columns = ['adult', 'backdrop_path', 'belongs_to_collection', 'budget', 'genres',
       'homepage', 'id', 'imdb_id', 'original_language', 'original_title',
       'overview', 'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'])

In [318]:
count = 0
fail_ids = []
start_timer = time.time()
for tmdb_id in movie_ids["id"][0:80].values:
    print(tmdb_id)
    try:
        with urllib.request.urlopen("http://api.themoviedb.org/3/movie/"+str(tmdb_id)+"?api_key="+api_key) as response:
            resp = response.readlines()[0]
        # just assuming it doesn't break!

        tmdb_df.loc[tmdb_id] = pd.DataFrame.from_dict(json.loads(resp), orient='index').T.values[0]
    except:
        print("failure at id = "+str(tmdb_id))
        fail_ids.append(tmdb_id)
        
    #X-RateLimit
    headers_dict = dict(response.headers.items())
    if 'X-RateLimit-Remaining' in headers_dict.keys():
        rate_limit_remaining = int(headers_dict['X-RateLimit-Remaining'])
        if rate_limit_remaining ==0:
            time.sleep(10)        
        
    count = count +1
    if (count % 40) == 0: # 40 api calls made
        finish_timer = time.time()
        seconds_passed = finish_timer-start_timer
        print("seconds_passed = ",seconds_passed)
        time.sleep( max(10 - seconds_passed,0) ) # respect the rate timer! no more than 40 requests every 10s
        
tmdb_df.to_msgpack( tmdb_dir+"tmdb_df"+".msgpack")

601
602
603
604
605
failure at id = 605
606
607
608
609
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638
639
640
641
642
seconds_passed =  67.9778881072998
643
644
646
647
648
649
650
651
652
653
654
655
657
658
659
660
661
662
663
664
665
666
667
668
669
670
671
672
673
674
675
676
677
678
679
680
681
682
685
686
seconds_passed =  135.65075874328613


In [209]:
np.array_split(movie_ids["id"][0:90].values,3)

[array([601, 602, 603, 604, 605, 606, 607, 608, 609, 612, 613, 614, 615,
        616, 617, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628,
        629, 630, 631, 632], dtype=int64),
 array([633, 634, 635, 636, 637, 638, 639, 640, 641, 642, 643, 644, 646,
        647, 648, 649, 650, 651, 652, 653, 654, 655, 657, 658, 659, 660,
        661, 662, 663, 664], dtype=int64),
 array([665, 666, 667, 668, 669, 670, 671, 672, 673, 674, 675, 676, 677,
        678, 679, 680, 681, 682, 685, 686, 687, 688, 689, 690, 691, 692,
        693, 694, 695, 696], dtype=int64)]

In [215]:
int(np.ceil( len(data)/40 ))

3

In [224]:
batch = tmdb_id_list[ tmdb_id_batch*40:((tmdb_id_batch+1)*40)   ]
print(batch)
URLS = [ "http://api.themoviedb.org/3/movie/"+str(x)+"?api_key="+api_key for x in batch]
URLS

[687 688 689 690 691 692 693 694 695 696]


['http://api.themoviedb.org/3/movie/687?api_key=0c8e2eac3198de39966b29221e7e740b',
 'http://api.themoviedb.org/3/movie/688?api_key=0c8e2eac3198de39966b29221e7e740b',
 'http://api.themoviedb.org/3/movie/689?api_key=0c8e2eac3198de39966b29221e7e740b',
 'http://api.themoviedb.org/3/movie/690?api_key=0c8e2eac3198de39966b29221e7e740b',
 'http://api.themoviedb.org/3/movie/691?api_key=0c8e2eac3198de39966b29221e7e740b',
 'http://api.themoviedb.org/3/movie/692?api_key=0c8e2eac3198de39966b29221e7e740b',
 'http://api.themoviedb.org/3/movie/693?api_key=0c8e2eac3198de39966b29221e7e740b',
 'http://api.themoviedb.org/3/movie/694?api_key=0c8e2eac3198de39966b29221e7e740b',
 'http://api.themoviedb.org/3/movie/695?api_key=0c8e2eac3198de39966b29221e7e740b',
 'http://api.themoviedb.org/3/movie/696?api_key=0c8e2eac3198de39966b29221e7e740b']

In [312]:
outFilePath = movie_ids_06_29_2019


movie_ids = pd.read_json(outFilePath, lines=True)
movie_ids.describe(include='all')

,adult,id,original_title,popularity,video
count,467223,467223.000000,467223,467223.000000,467223
unique,2,NaN,429550,NaN,2
top,False,NaN,Carmen,NaN,False
freq,443083,NaN,46,NaN,444777
mean,NaN,331788.498929,NaN,1.111966,NaN
std,NaN,175573.963916,NaN,2.037446,NaN
min,NaN,2.000000,NaN,0.000000,NaN
25%,NaN,191249.000000,NaN,0.600000,NaN
50%,NaN,346870.000000,NaN,0.600000,NaN
75%,NaN,481636.500000,NaN,0.778000,NaN


In [319]:
import concurrent.futures
import urllib.request

# URLS = #[some list of urls]
from multiprocessing import Pool
# from multiprocessing.dummy import Pool as ThreadPool


# Retrieve a json string
def load_json_str(url, timeout=60):
    try:
        response = urllib.request.urlopen(url, timeout=timeout)
        output = response.readlines()[0]
    except Exception as exc:
        print('%r generated an exception: %s' % (url, exc))    
#         output = None
        output = None
    return output

# tmdb_id_list = movie_ids["id"][0:90].values
all_movie_ids = movie_ids["id"].values
remaining_movie_ids = list( set(all_movie_ids).difference(set(list(tmdb_df.index))) )

tmdb_id_list  = remaining_movie_ids
# start_timer = time.time()


# Make the Pool of workers
pool = ThreadPool(2)
# Open the urls in their own threads
# and return the results

#close the pool and wait for the work to finish



for tmdb_id_batch in range(0, np.ceil( len(tmdb_id_list)/40 ).astype('int') ):
    batch = tmdb_id_list[ tmdb_id_batch*40:((tmdb_id_batch+1)*40)   ]
#     print(batch)
    
    URLS = [ "http://api.themoviedb.org/3/movie/"+str(x)+"?api_key="+api_key for x in batch]
    
    start_timer = time.time()
    # We can use a with statement to ensure threads are cleaned up promptly
#     with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
#         # Start the load operations and mark each future with its URL
#         future_to_url = {executor.submit(load_json_str, url, 60): url for url in URLS}
#         for future in concurrent.futures.as_completed(future_to_url):
#             url = future_to_url[future]
#             try:
#                 data = future.result()
#                 tmdb_df.loc[tmdb_id] = pd.DataFrame.from_dict(json.loads(data), orient='index').T.values[0]
#                 # do json processing here
#             except Exception as exc:
#                 print('%r generated an exception: %s' % (url, exc))
# #             else:
# #                 print('%r page is %d bytes' % (url, len(data)))
                
    
    results = pool.map(load_json_str, URLS)
#     for res in results:
#         tmdb_df.loc[tmdb_id] = pd.DataFrame.from_dict(json.loads(res), orient='index').T.values[0]
    
#     tmdb_df.loc[batch] =         pd.DataFrame.from_dict( [json.loads(res) for res in results] ).values
    
    df = pd.DataFrame.from_dict( [json.loads(res) for res in results if res is not None] )
    df.index = [batch[res_no] for res_no in range(0,len(results)) if results[res_no] is not None]
    tmdb_df = tmdb_df.append(df)
    
    finish_timer = time.time()
    duration = finish_timer-start_timer
    print(duration)
    time.sleep( max( 0,12 - duration ) )

    
pool.close()
pool.join()    

'http://api.themoviedb.org/3/movie/594155?api_key=0c8e2eac3198de39966b29221e7e740b' generated an exception: HTTP Error 404: Not Found
6.092348337173462
6.588376760482788
6.5143725872039795
6.742385625839233
6.537374019622803
6.7593865394592285
6.5153725147247314
6.653380632400513
6.700383424758911
6.735385417938232
6.595377206802368
7.004400730133057
7.254414796829224
6.696382999420166
5.3223042488098145


In [363]:
# outFilePath =  tmdb_dir + "production_company_ids_06_30_2019.json"
outFilePath = tmdb_dir + "movie_ids_06_30_2019.json"
namespace = "movie"

movie_ids = pd.read_json(outFilePath, lines=True)
movie_ids.describe(include='all')

,adult,id,original_title,popularity,video
count,467940,467940.000000,467940,467940.000000,467940
unique,2,NaN,430179,NaN,2
top,False,NaN,Carmen,NaN,False
freq,443700,NaN,46,NaN,445460
mean,NaN,332220.303394,NaN,1.122532,NaN
std,NaN,175782.794324,NaN,2.049486,NaN
min,NaN,2.000000,NaN,0.000000,NaN
25%,NaN,191600.750000,NaN,0.600000,NaN
50%,NaN,347267.500000,NaN,0.600000,NaN
75%,NaN,482253.250000,NaN,0.841000,NaN


In [351]:
# movie_ids

In [4]:
# outFilePath =  tmdb_dir + "production_company_ids_06_30_2019.json"
outFilePath = tmdb_dir + "person_ids_07_04_2019.json"
namespace = "person"

movie_ids = pd.read_json(outFilePath, lines=True)
movie_ids.describe(include='all')

,adult,id,name,popularity
count,1408860,1.408860e+06,1408860,1.408860e+06
unique,2,NaN,1299849,NaN
top,False,NaN,Led Sarmiento,NaN
freq,1373580,NaN,54,NaN
mean,NaN,1.502127e+06,NaN,6.545755e-01
std,NaN,6.360921e+05,NaN,4.153430e-01
min,NaN,1.000000e+00,NaN,0.000000e+00
25%,NaN,1.235013e+06,NaN,6.000000e-01
50%,NaN,1.615446e+06,NaN,6.000000e-01
75%,NaN,1.987396e+06,NaN,6.000000e-01


In [5]:
tmdb_df = pd.DataFrame(columns = [])
secondary_tmdb_dir = "e:\\Datasets\\themoviedb\\"
# tmdb_df = pd.read_msgpack( secondary_tmdb_dir+"person_ids_df_as_of_08_07_2019"+".msgpack")
# too big!
tmdb_df_indices = pd.read_pickle( secondary_tmdb_dir+"person_ids_df_as_of_08_07_2019_indices"+".pickle")

In [6]:
text_file = open(tmdb_dir+"my_api_key.txt", "r")
api_key = text_file.readlines()[0]

In [ ]:
import concurrent.futures
import urllib.request

# URLS = #[some list of urls]
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool


# Retrieve a json string
def load_json_str(url, timeout=60):
    try:
        response = urllib.request.urlopen(url, timeout=timeout)
        output = response.readlines()[0]
    except Exception as exc:
        print('%r generated an exception: %s' % (url, exc))    
#         output = None
        output = None
    return output

# tmdb_id_list = movie_ids["id"][0:90].values
all_movie_ids = movie_ids["id"].values
# remaining_movie_ids = list( set(all_movie_ids).difference(set(list(tmdb_df.index))) )
remaining_movie_ids = list( set(all_movie_ids).difference(set(list(tmdb_df_indices.values))) )
tmdb_id_list  = remaining_movie_ids
# start_timer = time.time()


# Make the Pool of workers
pool = ThreadPool(3)
# Open the urls in their own threads
# and return the results

#close the pool and wait for the work to finish



for tmdb_id_batch in range(0, np.ceil( len(tmdb_id_list)/40 ).astype('int') ):
    batch = tmdb_id_list[ tmdb_id_batch*40:((tmdb_id_batch+1)*40)   ]
#     print(batch)
    #URLS = [ "http://api.themoviedb.org/3/"+namespace+"/"+str(x)+"?api_key="+api_key for x in batch]
#     URLS = [ "https://api.themoviedb.org/3/"+namespace+"/"+str(x)+"?api_key="+api_key+"&language=en-US&append_to_response=alternative_titles%2Ccredits%2Cexternal_ids%2Ckeywords%2Crelease_dates" for x in batch]
    URLS = [ "https://api.themoviedb.org/3/"+namespace+"/"+str(x)+"?api_key="+api_key+"&language=en-US&append_to_response=images%2Ccredits%2Cexternal_ids" for x in batch]    
# https://api.themoviedb.org/3/movie/687?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=alternative_titles%2Ccredits%2Cexternal_ids%2Ckeywords%2Crelease_dates
    
    
    start_timer = time.time()
               
    
    results = pool.map(load_json_str, URLS)
    
    df = pd.DataFrame.from_dict( [json.loads(res) for res in results if res is not None] )
    df.index = [batch[res_no] for res_no in range(0,len(results)) if results[res_no] is not None]
    tmdb_df = tmdb_df.append(df)
    
    finish_timer = time.time()
    duration = finish_timer-start_timer
    print(duration)
    time.sleep( max( 0,11 - duration ) )

    
pool.close()
pool.join()    

'https://api.themoviedb.org/3/person/2169279?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=images%2Ccredits%2Cexternal_ids' generated an exception: HTTP Error 404: Not Found'https://api.themoviedb.org/3/person/2177505?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=images%2Ccredits%2Cexternal_ids' generated an exception: HTTP Error 404: Not Found'https://api.themoviedb.org/3/person/2121866?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=images%2Ccredits%2Cexternal_ids' generated an exception: HTTP Error 404: Not Found


'https://api.themoviedb.org/3/person/2177815?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=images%2Ccredits%2Cexternal_ids' generated an exception: HTTP Error 404: Not Found
'https://api.themoviedb.org/3/person/2134598?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=images%2Ccredits%2Cexternal_ids' generated an exception: HTTP Error 

'https://api.themoviedb.org/3/person/2311507?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=images%2Ccredits%2Cexternal_ids' generated an exception: HTTP Error 429: 
'https://api.themoviedb.org/3/person/2311499?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=images%2Ccredits%2Cexternal_ids' generated an exception: HTTP Error 429: 'https://api.themoviedb.org/3/person/2311503?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=images%2Ccredits%2Cexternal_ids' generated an exception: HTTP Error 429: 

'https://api.themoviedb.org/3/person/2311515?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=images%2Ccredits%2Cexternal_ids' generated an exception: HTTP Error 429: 
'https://api.themoviedb.org/3/person/2311504?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=images%2Ccredits%2Cexternal_ids' generated an exception: HTTP Error 429: 
'https://api.themoviedb.org/3/

'https://api.themoviedb.org/3/person/1415259?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=images%2Ccredits%2Cexternal_ids' generated an exception: HTTP Error 404: Not Found
'https://api.themoviedb.org/3/person/2352747?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=images%2Ccredits%2Cexternal_ids' generated an exception: HTTP Error 404: Not Found'https://api.themoviedb.org/3/person/2352743?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=images%2Ccredits%2Cexternal_ids' generated an exception: HTTP Error 404: Not Found

'https://api.themoviedb.org/3/person/1434375?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=images%2Ccredits%2Cexternal_ids' generated an exception: HTTP Error 404: Not Found
'https://api.themoviedb.org/3/person/2352748?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=images%2Ccredits%2Cexternal_ids' generated an exception: HTTP Error 

'https://api.themoviedb.org/3/person/1672380?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=images%2Ccredits%2Cexternal_ids' generated an exception: HTTP Error 429: 'https://api.themoviedb.org/3/person/1672384?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=images%2Ccredits%2Cexternal_ids' generated an exception: HTTP Error 429: 

'https://api.themoviedb.org/3/person/1672381?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=images%2Ccredits%2Cexternal_ids' generated an exception: HTTP Error 429: 
'https://api.themoviedb.org/3/person/1672377?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=images%2Ccredits%2Cexternal_ids' generated an exception: HTTP Error 429: 'https://api.themoviedb.org/3/person/1672385?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=images%2Ccredits%2Cexternal_ids' generated an exception: HTTP Error 429: 

'https://api.themoviedb.org/3/

8.67949628829956
9.359535455703735
8.750500440597534
8.70649790763855
9.073518991470337
8.54248857498169
8.656495094299316
8.9395112991333
11.42465353012085
'https://api.themoviedb.org/3/person/1677897?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=images%2Ccredits%2Cexternal_ids' generated an exception: HTTP Error 429: 
'https://api.themoviedb.org/3/person/1677898?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=images%2Ccredits%2Cexternal_ids' generated an exception: HTTP Error 429: 
8.810503959655762
8.999515056610107
8.683496713638306
9.953569412231445
8.77450180053711
8.607492446899414
8.7545006275177
8.537488222122192
8.743500471115112
9.06151819229126
11.067633152008057
'https://api.themoviedb.org/3/person/1678364?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=images%2Ccredits%2Cexternal_ids' generated an exception: HTTP Error 429: 
8.397480487823486
8.410480737686157
8.477484941482544
8.4114811420

...

'https://api.themoviedb.org/3/person/1950994?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=images%2Ccredits%2Cexternal_ids' generated an exception: HTTP Error 429: 
'https://api.themoviedb.org/3/person/1951008?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=images%2Ccredits%2Cexternal_ids' generated an exception: HTTP Error 429: 
'https://api.themoviedb.org/3/person/1950991?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=images%2Ccredits%2Cexternal_ids' generated an exception: HTTP Error 429: 
'https://api.themoviedb.org/3/person/1950995?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=images%2Ccredits%2Cexternal_ids' generated an exception: HTTP Error 429: 
'https://api.themoviedb.org/3/person/1951010?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=images%2Ccredits%2Cexternal_ids' generated an exception: HTTP Error 429: 
8.496486186981201
8.5954916477

'https://api.themoviedb.org/3/person/1954651?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=images%2Ccredits%2Cexternal_ids' generated an exception: HTTP Error 429: 'https://api.themoviedb.org/3/person/1954642?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=images%2Ccredits%2Cexternal_ids' generated an exception: HTTP Error 429: 

'https://api.themoviedb.org/3/person/1954656?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=images%2Ccredits%2Cexternal_ids' generated an exception: HTTP Error 429: 
'https://api.themoviedb.org/3/person/1954652?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=images%2Ccredits%2Cexternal_ids' generated an exception: HTTP Error 429: 
'https://api.themoviedb.org/3/person/1954643?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=images%2Ccredits%2Cexternal_ids' generated an exception: HTTP Error 429: 
'https://api.themoviedb.org/3/

'https://api.themoviedb.org/3/person/1957791?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=images%2Ccredits%2Cexternal_ids' generated an exception: HTTP Error 429: 
'https://api.themoviedb.org/3/person/1957783?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=images%2Ccredits%2Cexternal_ids' generated an exception: HTTP Error 429: 
'https://api.themoviedb.org/3/person/1957787?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=images%2Ccredits%2Cexternal_ids' generated an exception: HTTP Error 429: 
8.586491107940674
8.79550313949585
8.814504146575928
8.888508558273315
9.249528884887695
8.745500087738037
8.626493453979492
8.821504354476929
8.95951247215271
11.292645931243896
'https://api.themoviedb.org/3/person/1958254?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=images%2Ccredits%2Cexternal_ids' generated an exception: HTTP Error 429: 
8.742500066757202
8.841505527496338
8.7274990

'https://api.themoviedb.org/3/person/1963351?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=images%2Ccredits%2Cexternal_ids' generated an exception: HTTP Error 429: 
'https://api.themoviedb.org/3/person/1963342?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=images%2Ccredits%2Cexternal_ids' generated an exception: HTTP Error 429: 
'https://api.themoviedb.org/3/person/1963352?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=images%2Ccredits%2Cexternal_ids' generated an exception: HTTP Error 429: 
'https://api.themoviedb.org/3/person/1963344?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=images%2Ccredits%2Cexternal_ids' generated an exception: HTTP Error 429: 'https://api.themoviedb.org/3/person/1963348?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=images%2Ccredits%2Cexternal_ids' generated an exception: HTTP Error 429: 

'https://api.themoviedb.org/3/

'https://api.themoviedb.org/3/person/1969377?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=images%2Ccredits%2Cexternal_ids' generated an exception: HTTP Error 429: 
'https://api.themoviedb.org/3/person/1969378?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=images%2Ccredits%2Cexternal_ids' generated an exception: HTTP Error 429: 
9.114521265029907
11.127636432647705
11.616664409637451
10.597606182098389
11.46565580368042
11.037631273269653
10.482599258422852
10.012573003768921
10.394594430923462
10.684611320495605
11.9856858253479
10.567604541778564
11.339648485183716
10.77561616897583
9.934568166732788
10.138579845428467
10.17458200454712
11.025630712509155
9.960569620132446
9.994571685791016
9.711555480957031
13.545774698257446
11.503658056259155
11.796674489974976
11.146637439727783
11.592663049697876
11.033630847930908
11.50665807723999
10.83161973953247
9.622550249099731
9.66955304145813
34.20495629310608
10.553603410720825
9

In [ ]:
# tmdb_df.index.to_pickle( secondary_tmdb_dir+"person_ids_df_as_of_08_07_2019_indices"+".pickle")

In [21]:
# tmdb_df.reset_index()["index"].to_pickle( secondary_tmdb_dir+"person_ids_df_as_of_08_07_2019_indices"+".pickle")

In [24]:
tmdb_df_indices.values

array([      1,       2,       3, ..., 1669705, 1669706, 1669707],
      dtype=int64)

In [25]:
tmdb_df.index

Int64Index([      1,       2,       3,       4,       5,       6,       7,
                  8,       9,      10,
            ...
            1669698, 1669699, 1669700, 1669701, 1669702, 1669703, 1669704,
            1669705, 1669706, 1669707],
           dtype='int64', length=1003261)

In [16]:
len(remaining_movie_ids)

729266

In [17]:
len(all_movie_ids)

1408860

In [9]:
gc.collect()

0

In [367]:
# load_json_str("https://api.themoviedb.org/3/movie/687?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=alternative_titles%2Ccredits%2Cexternal_ids%2Ckeywords%2Crelease_dates")

In [368]:
# res = load_json_str("http://api.themoviedb.org/3/collection/645?api_key=0c8e2eac3198de39966b29221e7e740b")


In [10]:
# pd.DataFrame.from_dict( json.loads(res) )
# ~7 hrs -> 86,000 rows out of 450,000
# 36.4 -> another 30hrs
# 1/5 -> 300MB -> 1.5Gb (maybe save to other hd!)
# ram not an issue - 200MB for tmbd_df


def pretty_print(df):
    return display( HTML( df.to_html().replace("\\n","<br>") ) )

def sizeof_fmt(num, suffix='B'):
    ''' By Fred Cirera, after https://stackoverflow.com/a/1094933/1870254'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)


def memory_use(locs = locals().items()):
    gc.collect()
    # locals().items()
    for name, size in sorted(((name, sys.getsizeof(value)) for name,value in locs),
                             key= lambda x: -x[1])[:10]:
        print("{:>30}: {:>8}".format(name,sizeof_fmt(size)))

In [11]:
import sys
memory_use(locs = locals().items())

                       tmdb_df:   1.2GiB
                     movie_ids: 121.5MiB
           remaining_movie_ids:   3.5MiB
                  tmdb_id_list:   3.5MiB
                            df:  50.9KiB
                          _iii:   2.2KiB
                           _i8:   2.2KiB
                  XGBRegressor:   2.0KiB
                            __:   1.6KiB
                            _5:   1.6KiB


In [12]:
import sys
memory_use(locs = globals().items())

                       tmdb_df:   1.2GiB
                     movie_ids: 121.5MiB
           remaining_movie_ids:   3.5MiB
                  tmdb_id_list:   3.5MiB
                            df:  50.9KiB
                           _i8:   2.2KiB
                  XGBRegressor:   2.0KiB
                            __:   1.6KiB
                            _5:   1.6KiB
                          HTML:   1.0KiB


In [40]:
del _,_32

In [20]:
gc.collect()

38

In [19]:
# tmdb_df.to_msgpack( secondary_tmdb_dir+"person_ids_df_as_of_08_07_2019"+".msgpack")

In [12]:
tmdb_df.shape
# 310,000 out of 1,400,000
# 1/5-1/4
# 1.7Gb on disk, 400Mb in memory

# lot of memory leakage

(679594, 17)

In [48]:
tmdb_df2.shape

(310303, 17)

In [45]:
# tmdb_df2 = pd.read_msgpack( secondary_tmdb_dir+"person_ids_df_as_of_06_07_2019"+".msgpack")

In [21]:
tmdb_df.dtypes

adult                      bool
also_known_as            object
biography                object
birthday                 object
credits                  object
deathday                 object
external_ids             object
gender                    int64
homepage                 object
id                        int64
images                   object
imdb_id                  object
known_for_department     object
name                     object
place_of_birth           object
popularity              float64
profile_path             object
dtype: object

In [327]:
# (1400000/4)/3600
# getting all people data -> 97hrs!

In [22]:
len(all_movie_ids)

1408860

In [23]:
tmdb_df.shape[0]

1002234

In [36]:
tmdb_dir

'..\\Datasets\\\\themoviedb\\'

In [328]:
# #X-RateLimit
# headers_dict = dict(response.headers.items())
# if 'X-RateLimit-Remaining' in headers_dict.keys():
#     rate_limit_remaining = int(headers_dict['X-RateLimit-Remaining'])
#     if rate_limit_remaining ==0:
#         time.sleep(10)


In [329]:
# headers_dict = dict(response.headers.items())
# headers_dict

In [321]:
# tmdb_df.to_msgpack( tmdb_dir+"tmdb_df_final"+".msgpack")

In [331]:
# ((467223/40)*10) / 3600

In [330]:
# 1200 -> ~1MB
# 467223/1200
# -> 400MB

In [ ]:
# 32 hours starting ~ 17ish Wednesday -> 1am Friday morning ... some fixes later, done on Sunday!